In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import requests
import os
import scipy.signal
import scipy.stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.model_selection import KFold
from dateutil.relativedelta import relativedelta
import time
from dateutil.tz import gettz
from datetime import timedelta
import calendar
import datetime
# from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
# import xgboost as xgb
%matplotlib inline
pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 50000)


chi = [6.635,9.21,11.345,13.277,15.086,16.812,18.475,20.09,21.666,23.209,24.725,26.217,27.688,29.141,30.578,32,33.409,34.805,36.191,37.566,38.932,40.289,41.638,42.98,44.314,45.642,46.963,48.278,49.588,50.892,52.191,53.486,54.776,56.061,57.342,58.619,59.893,61.162,62.428,63.691,64.95,66.206,67.459,68.71,69.957,71.201,72.443,73.683,74.919,76.154,77.386,78.616,79.843,81.069,82.292,83.513,84.733,85.95,87.166,88.379,89.591,90.802,92.01,93.217,94.422,95.626,96.828,98.028,99.228,100.425,101.621,102.816,104.01,105.202,106.393,107.583,108.771,109.958,111.144,112.329,113.512,114.695,115.876,117.057,118.236,119.414,120.591,121.767,122.942,124.116,125.289,126.462,127.633,128.803,129.973,131.141,132.309,133.476,134.642,135.807,136.971,138.134,139.297,140.459,141.62,142.78,143.94,145.099,146.257,147.414,148.571,149.727,150.882,152.037,153.191,154.344,155.496,156.648,157.8,158.95,160.1,161.25,162.398,163.546,164.694,165.841,166.987,168.133,169.278,170.423,171.567,172.711,173.854,174.996,176.138,177.28,178.421,179.561,180.701,181.84,182.979,184.118,185.256,186.393,187.53,188.666,189.802,190.938,192.073,193.208,194.342,195.476,196.609,197.742,198.874,200.006,201.138,202.269,203.4,204.53,205.66,206.79,207.919,209.047,210.176,211.304,212.431,213.558,214.685,215.812,216.938,218.063,219.189,220.314,221.438,222.563,223.687,224.81,225.933,227.056,228.179,229.301,230.423,231.544,232.665,233.786,234.907,236.027,237.147,238.266,239.386,240.505,241.623,242.742,243.86,244.977,246.095,247.212,248.329,249.445
]

step = 50

In [2]:
## Preprocess data for hairdryer and build corresponding model
hairdryer = pd.read_csv('disagData/emily/hairdryerPhilips_power.csv')
hairdryer.drop('ts',axis=1, inplace=True)
hairdryer['label'] = np.nan
hairdryer['label'].iloc[:510]=0
hairdryer['label'].iloc[510:6788]=1
hairdryer['label'].iloc[6788:]=0

## Preprocess data for tv and build corresponding model
telv = pd.read_csv('disagData/emily/LG_power.csv')
telv.drop('ts',axis=1, inplace=True)
telv['label'] = 1


## Preprocess data for laptop and build corresponding model
laptop = pd.read_csv('disagData/emily/laptop_power.csv')
laptop.drop('ts',axis=1, inplace=True)
laptop['label'] = 1

## Preprocess data for fridge and build corresponding model
fridge = pd.read_csv('disagData/emily/fridge_power.csv')
fridge.drop('ts',axis=1, inplace=True)
fridge['label'] = 1


In [3]:
## Preprocess data for hairdryer and build corresponding model

hd = pd.concat([fridge,telv,laptop,hairdryer])
hd['label'].iloc[:fridge.shape[0]+laptop.shape[0]+telv.shape[0]]=0

cols = ['pwrA','apwrA']

hairdryer_pr = pd.DataFrame([])
for col in cols:
    hairdryer_pr[col+'_mean'] = hd[col].groupby(np.arange(len(hd))//step).mean()
    hairdryer_pr[col+'_std'] = hd[col].groupby(np.arange(len(hd))//step).std()
    hairdryer_pr[col+'_min'] = hd[col].groupby(np.arange(len(hd))//step).min()
    hairdryer_pr[col+'_max'] = hd[col].groupby(np.arange(len(hd))//step).max()
hairdryer_pr['label'] = hd['label'].groupby(np.arange(len(hd))//step).agg(lambda x:x.value_counts().index[0])

df = hairdryer_pr.copy()

# split dataframe into feature set and labels
y = df['label'].values
X_hd = df.drop('label',axis=1)
# X = preprocessing.MinMaxScaler().fit_transform(X)

# split into training and test sets and train the model

# mdl_hd = KNeighborsClassifier(n_neighbors=5)
mdl_hd = DecisionTreeClassifier(random_state=0,max_depth=50)
X_train, X_test, y_train, y_test = train_test_split(X_hd, y,
                                                    stratify=y, 
                                                    test_size=0.25)

mdl_hd = mdl_hd.fit(X_train,y_train)

y_pred = mdl_hd.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

1.0


In [4]:
## Preprocess data for tv and build corresponding model


tv = pd.concat([fridge,laptop,hairdryer,telv])
tv['label'].iloc[:fridge.shape[0]+laptop.shape[0]+hairdryer.shape[0]]=0
tv.reset_index(inplace=True, drop=True)

cols = ['pwrA','apwrA']

tv_pr = pd.DataFrame([])
for col in cols:
    tv_pr[col+'_mean'] = tv[col].groupby(np.arange(len(tv))//step).mean()
    tv_pr[col+'_std'] = tv[col].groupby(np.arange(len(tv))//step).std()
    tv_pr[col+'_min'] = tv[col].groupby(np.arange(len(tv))//step).min()
    tv_pr[col+'_max'] = tv[col].groupby(np.arange(len(tv))//step).max()
tv_pr['label'] = tv['label'].groupby(np.arange(len(tv))//step).agg(lambda x:x.value_counts().index[0])

df = tv_pr.copy()

# split dataframe into feature set and labels
y = df['label'].values
X_tv = df.drop('label',axis=1)
# X = preprocessing.MinMaxScaler().fit_transform(X)

# split into training and test sets and train the model

# mdl_tv = KNeighborsClassifier(n_neighbors=5)
mdl_tv = DecisionTreeClassifier(random_state=0,max_depth=50)

X_train, X_test, y_train, y_test = train_test_split(X_tv, y,
                                                    stratify=y, 
                                                    test_size=0.25)

mdl_tv = mdl_tv.fit(X_train,y_train)

y_pred = mdl_tv.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

1.0


In [5]:
## Preprocess data for tv and build corresponding model

lp = pd.concat([fridge,telv,hairdryer,laptop])
lp['label'].iloc[:fridge.shape[0]+hairdryer.shape[0]+telv.shape[0]]=0
lp.reset_index(inplace=True, drop=True)

cols = ['pwrA','apwrA']

laptop_pr = pd.DataFrame([])
for col in cols:
    laptop_pr[col+'_mean'] = lp[col].groupby(np.arange(len(lp))//step).mean()
    laptop_pr[col+'_std'] = lp[col].groupby(np.arange(len(lp))//step).std()
    laptop_pr[col+'_min'] = lp[col].groupby(np.arange(len(lp))//step).min()
    laptop_pr[col+'_max'] = lp[col].groupby(np.arange(len(lp))//step).max()
laptop_pr['label'] = lp['label'].groupby(np.arange(len(lp))//step).agg(lambda x:x.value_counts().index[0])

df = laptop_pr.copy()

# split dataframe into feature set and labels
y = df['label'].values
X_lp = df.drop('label',axis=1)
# X = preprocessing.MinMaxScaler().fit_transform(X)

# split into training and test sets and train the model

# mdl_tv = KNeighborsClassifier(n_neighbors=5)
mdl_lp = DecisionTreeClassifier(random_state=0,max_depth=50)

X_train, X_test, y_train, y_test = train_test_split(X_lp, y,
                                                    stratify=y, 
                                                    test_size=0.25)

mdl_lp = mdl_lp.fit(X_train,y_train)

y_pred = mdl_lp.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

1.0


In [6]:
## Preprocess data for tv and build corresponding model

fr = pd.concat([telv,hairdryer,laptop,fridge])
fr['label'].iloc[:laptop.shape[0]+hairdryer.shape[0]+telv.shape[0]]=0
fr.reset_index(inplace=True, drop=True)

cols = ['pwrA','apwrA']

fridge_pr = pd.DataFrame([])
for col in cols:
    fridge_pr[col+'_mean'] = fr[col].groupby(np.arange(len(fr))//step).mean()
    fridge_pr[col+'_std'] = fr[col].groupby(np.arange(len(fr))//step).std()
    fridge_pr[col+'_min'] = fr[col].groupby(np.arange(len(fr))//step).min()
    fridge_pr[col+'_max'] = fr[col].groupby(np.arange(len(fr))//step).max()
fridge_pr['label'] = fr['label'].groupby(np.arange(len(fr))//step).agg(lambda x:x.value_counts().index[0])

df = fridge_pr.copy()

# split dataframe into feature set and labels
y = df['label'].values
X_fr = df.drop('label',axis=1)
# X = preprocessing.MinMaxScaler().fit_transform(X)

# split into training and test sets and train the model

# mdl_tv = KNeighborsClassifier(n_neighbors=5)
mdl_fr = DecisionTreeClassifier(random_state=0,max_depth=50)

X_train, X_test, y_train, y_test = train_test_split(X_fr, y,
                                                    stratify=y, 
                                                    test_size=0.25)

mdl_fr = mdl_fr.fit(X_train,y_train)

y_pred = mdl_fr.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

1.0


In [7]:
# Read tv and hairdryer data
df = pd.read_csv('disagData/emily/tv_hairdryer_power.csv')
df.drop('ts',axis=1, inplace=True)

df['totalpwr'] =df['pwrA']+df['rpwrA']* 1j 


# throw out the last elements, in order to have a round division 

df = df.iloc[:-np.mod(df.shape[0],step)]

active = df['pwrA'].as_matrix()
active= np.reshape(active,(-1,step))

totalpwr = df['totalpwr'].as_matrix()
totalpwr = np.reshape(totalpwr,(-1,step))

events=np.array([])
events = np.append(events,0)

for i in range(0,totalpwr.shape[0]-1):
    lgof = np.absolute(np.sum((totalpwr[i+1]-totalpwr[i])**2/totalpwr[i]))
    if lgof> chi[step-2]:
        diff = np.absolute(np.mean(active[i+1])-np.mean(active[i]))
        if diff>15:
            lgof=1
        else:
            lgof=0
    else:
        lgof= 0
    events = np.append(events,lgof)

cols = ['pwrA','apwrA']

df_pr = pd.DataFrame([])
for col in cols:
    df_pr[col+'_mean'] = df[col].groupby(np.arange(len(df))//step).mean()
    df_pr[col+'_std'] = df[col].groupby(np.arange(len(df))//step).std()
    df_pr[col+'_min'] = df[col].groupby(np.arange(len(df))//step).min()
    df_pr[col+'_max'] = df[col].groupby(np.arange(len(df))//step).max()
    
df_pr['events']=events

df_pr

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


,pwrA_mean,pwrA_std,pwrA_min,pwrA_max,apwrA_mean,apwrA_std,apwrA_min,apwrA_max,events
0,16.451398,5.724898,10.7325,25.5763,31.656836,8.854646,22.2740,43.9206,0.0
1,19.840214,2.885002,14.2990,22.7363,34.616028,4.827673,25.8735,39.7762,0.0
2,22.326194,0.122680,21.9933,22.6373,39.396108,0.198017,39.1323,39.7927,0.0
3,22.666984,0.350213,21.9438,23.1326,39.750446,0.528833,38.9506,40.3871,0.0
4,22.609856,0.140563,22.3566,22.8849,39.998122,0.140828,39.7762,40.2385,0.0
5,22.513110,0.101413,22.3566,22.7198,39.702886,0.152618,39.4790,39.9909,0.0
6,21.905808,0.656696,20.5238,22.5382,38.903398,0.852045,36.9032,39.6771,0.0
7,20.194562,0.259459,19.7147,20.5403,36.070032,0.366007,35.4667,36.8041,0.0
8,20.111998,0.159954,19.8798,20.4412,35.824016,0.223919,35.5493,36.4079,0.0
9,20.458080,0.307412,19.7808,20.8210,36.602034,0.416543,35.5823,37.0023,0.0


In [8]:
fev = 0

for i in range(1,df_pr.shape[0]):
    df2 = df_pr.copy()
    if fev>0:
        df2['pwrA_mean'].iloc[i] = np.abs(df2['pwrA_mean'].iloc[i] - prev_pwrA_mean)
        df2['pwrA_std'].iloc[i] = np.abs(df2['pwrA_std'].iloc[i] - prev_pwrA_std)
        df2['pwrA_min'].iloc[i] = np.abs(df2['pwrA_min'].iloc[i] - prev_pwrA_min)
        df2['pwrA_max'].iloc[i] = np.abs(df2['pwrA_max'].iloc[i] - prev_pwrA_max)
        df2['apwrA_mean'].iloc[i] = np.abs(df2['apwrA_mean'].iloc[i] - prev_apwrA_mean)
        df2['apwrA_std'].iloc[i] = np.abs(df2['apwrA_std'].iloc[i] - prev_apwrA_std)
        df2['apwrA_min'].iloc[i] = np.abs(df2['apwrA_min'].iloc[i] - prev_apwrA_min)
        df2['apwrA_max'].iloc[i] = np.abs(df2['apwrA_max'].iloc[i] - prev_apwrA_max)
        
    if (df_pr['events'].iloc[i]==1.0 and df_pr['events'].iloc[i-1] == 0.0):
        fev = 1
        prev_pwrA_mean = np.abs(df2['pwrA_mean'].iloc[i-1])# - prev_pwrA_mean)
        prev_pwrA_std = np.abs(df2['pwrA_std'].iloc[i-1]) #- prev_pwrA_std)
        prev_pwrA_min = np.abs(df2['pwrA_min'].iloc[i-1]) #- prev_pwrA_min)
        prev_pwrA_max = np.abs(df2['pwrA_max'].iloc[i-1]) #- prev_pwrA_max)
        prev_apwrA_mean = np.abs(df2['apwrA_mean'].iloc[i-1]) #- prev_apwrA_mean)
        prev_apwrA_std = np.abs(df2['apwrA_std'].iloc[i-1]) #- prev_apwrA_std)
        prev_apwrA_min = np.abs(df2['apwrA_min'].iloc[i-1]) #- prev_apwrA_min)
        prev_apwrA_max = np.abs(df2['apwrA_max'].iloc[i-1]) #- prev_apwrA_max)
#         print(prev_pwrA_mean,prev_pwrA_std,prev_pwrA_min,prev_pwrA_max,prev_apwrA_mean,prev_apwrA_std,prev_apwrA_min,prev_apwrA_max)

        
#     if (df_pr['events'].iloc[i]==0.0 and df_pr['events'].iloc[i-1]==1.0):
#         fev = 1
#         print('found event')
           
        
        
            
    X = df2.iloc[i][:-1].values.reshape(1,-1)
#     print('X:',X)
    if mdl_hd.predict(X)>0:print('hairdryer')
    if mdl_lp.predict(X)>0:print('laptop')
    if mdl_tv.predict(X)>0:print('tv')
    if mdl_fr.predict(X)>0:print('fridge')
    

tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
hairdryer
fridge
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer
hairdryer


In [199]:
# Read fridge and tv data
df = pd.read_csv('disagData/emily/fridge_tv_power.csv')
df.drop('ts',axis=1, inplace=True)

df['totalpwr'] =df['pwrA']+df['rpwrA']* 1j 


# throw out the last elements, in order to have a round division 

df = df.iloc[:-np.mod(df.shape[0],step)]

active = df['pwrA'].as_matrix()
active= np.reshape(active,(-1,step))

totalpwr = df['totalpwr'].as_matrix()
totalpwr = np.reshape(totalpwr,(-1,step))

events=np.array([])
events = np.append(events,0)

for i in range(0,totalpwr.shape[0]-1):
    lgof = np.absolute(np.sum((totalpwr[i+1]-totalpwr[i])**2/totalpwr[i]))
    if lgof> chi[step-2]:
        diff = np.absolute(np.mean(active[i+1])-np.mean(active[i]))
        if diff>15:
            lgof=1
        else:
            lgof=0
    else:
        lgof= 0
    events = np.append(events,lgof)

cols = ['pwrA','apwrA']

df_pr = pd.DataFrame([])
for col in cols:
    df_pr[col+'_mean'] = df[col].groupby(np.arange(len(df))//step).mean()
    df_pr[col+'_std'] = df[col].groupby(np.arange(len(df))//step).std()
    df_pr[col+'_min'] = df[col].groupby(np.arange(len(df))//step).min()
    df_pr[col+'_max'] = df[col].groupby(np.arange(len(df))//step).max()
    
df_pr['events']=events

df_pr

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


,pwrA_mean,pwrA_std,pwrA_min,pwrA_max,apwrA_mean,apwrA_std,apwrA_min,apwrA_max,events
0,70.339646,3.485623,65.0553,76.3987,83.125170,3.435410,78.4957,89.7400,0.0
1,76.816784,0.398089,74.5494,77.1417,90.729366,0.297110,89.8886,91.0774,0.0
2,64.732346,2.200297,61.9181,73.0799,78.882038,3.368287,75.8538,89.2612,0.0
3,59.697994,1.758826,57.2949,61.7530,73.173678,1.772306,70.3225,75.6887,0.0
4,57.404544,0.104894,57.2123,57.5921,70.625300,0.128445,70.4215,70.8674,0.0
5,57.204094,0.163633,56.9317,57.4930,70.494526,0.206264,70.1904,70.9004,0.0
6,56.705778,0.245962,56.3372,57.1298,70.109470,0.279221,69.5464,70.6032,0.0
7,57.358642,0.369327,56.7170,57.9719,70.492548,0.386096,69.6950,71.2801,0.0
8,58.869442,0.729821,57.8068,59.9202,71.727278,0.568792,70.9169,72.6836,0.0
9,59.878296,0.095659,59.6230,60.0028,72.695184,0.181310,72.3039,72.9808,0.0


In [209]:
# prev_pwrA_mean = df_pr['pwrA_mean'].iloc[0]
# prev_pwrA_std = df_pr['pwrA_std'].iloc[0]
# prev_pwrA_min = df_pr['pwrA_min'].iloc[0]
# prev_pwrA_max = df_pr['pwrA_max'].iloc[0]
# prev_apwrA_mean = df_pr['apwrA_mean'].iloc[0]
# prev_apwrA_std = df_pr['apwrA_std'].iloc[0]
# prev_apwrA_min = df_pr['apwrA_min'].iloc[0]
# prev_apwrA_max = df_pr['apwrA_max'].iloc[0]

# print(prev_pwrA_mean,prev_pwrA_std,prev_pwrA_min,prev_pwrA_max,prev_apwrA_mean,prev_apwrA_std,prev_apwrA_min,prev_apwrA_max)

fev = 0

for i in range(1,df_pr.shape[0]):
    df2 = df_pr.copy()
    if fev>0:
        df2['pwrA_mean'].iloc[i] = np.abs(df2['pwrA_mean'].iloc[i] - prev_pwrA_mean)
        df2['pwrA_std'].iloc[i] = np.abs(df2['pwrA_std'].iloc[i] - prev_pwrA_std)
        df2['pwrA_min'].iloc[i] = np.abs(df2['pwrA_min'].iloc[i] - prev_pwrA_min)
        df2['pwrA_max'].iloc[i] = np.abs(df2['pwrA_max'].iloc[i] - prev_pwrA_max)
        df2['apwrA_mean'].iloc[i] = np.abs(df2['apwrA_mean'].iloc[i] - prev_apwrA_mean)
        df2['apwrA_std'].iloc[i] = np.abs(df2['apwrA_std'].iloc[i] - prev_apwrA_std)
        df2['apwrA_min'].iloc[i] = np.abs(df2['apwrA_min'].iloc[i] - prev_apwrA_min)
        df2['apwrA_max'].iloc[i] = np.abs(df2['apwrA_max'].iloc[i] - prev_apwrA_max)
        
    if df_pr['events'].iloc[i]==1.0:
        prev_pwrA_mean = np.abs(df2['pwrA_mean'].iloc[i-1])# - prev_pwrA_mean)
        prev_pwrA_std = np.abs(df2['pwrA_std'].iloc[i-1]) #- prev_pwrA_std)
        prev_pwrA_min = np.abs(df2['pwrA_min'].iloc[i-1]) #- prev_pwrA_min)
        prev_pwrA_max = np.abs(df2['pwrA_max'].iloc[i-1]) #- prev_pwrA_max)
        prev_apwrA_mean = np.abs(df2['apwrA_mean'].iloc[i-1]) #- prev_apwrA_mean)
        prev_apwrA_std = np.abs(df2['apwrA_std'].iloc[i-1]) #- prev_apwrA_std)
        prev_apwrA_min = np.abs(df2['apwrA_min'].iloc[i-1]) #- prev_apwrA_min)
        prev_apwrA_max = np.abs(df2['apwrA_max'].iloc[i-1]) #- prev_apwrA_max)
#         print(prev_pwrA_mean,prev_pwrA_std,prev_pwrA_min,prev_pwrA_max,prev_apwrA_mean,prev_apwrA_std,prev_apwrA_min,prev_apwrA_max)

        
    if (df_pr['events'].iloc[i]==0.0 and df_pr['events'].iloc[i-1]==1.0):
        fev = 1
        print('found event')
           
        
        
            
    X = df2.iloc[i][:-1].values.reshape(1,-1)
#     print('X:',X)
    if mdl_hd.predict(X)>0:print('hairdryer')
    if mdl_lp.predict(X)>0:print('laptop')
    if mdl_tv.predict(X)>0:print('tv')
    if mdl_fr.predict(X)>0:print('fridge')
    

fridge
fridge
fridge
fridge
fridge
fridge
fridge
fridge
fridge
fridge
fridge
fridge
fridge
fridge
fridge
fridge
fridge
found event
fridge
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
